## Grepping functions from the vulnerability context of the file.

In [39]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import lizard
import subprocess as sub
from pylibsrcml import srcml
import os
import xml.etree.ElementTree as et 

Index(['File', 'Line', 'Column', 'DefaultLevel', 'Level', 'Category', 'Name',
       'Warning', 'Suggestion', 'Note', 'CWEs', 'Context', 'Fingerprint',
       'ToolVersion', 'RuleId', 'HelpUri'],
      dtype='object')

In [41]:
# Function Under Construction:
def srcML_funs(file):
    """ finds function blocks of the given file
    """ 
    fun_ptn = "string(//src:function)"
    funblk_ptn = "string((//src:function/src:name))"
    # file_ptn = "string(//src:unit/@filename)" 

    # cmd = sub.Popen(["srcml", "--xpath", fun_ptn, file], stderr=sub.STDOUT)
    # out, err = cmd.communicate()
    cmd = ["srcml", "--xpath", funblk_ptn, xml_file]
    process = sub.Popen(cmd, stderr=sub.STDOUT)
    return process

file = '../' + df.File[0]
xml_file = "a.cpp.xml"

file_str = ''
with open(file) as f:
    file_str = f.read()

# Fetching the functions which have given line context/statement.

In [43]:
import itertools
    
    
def file2metrics(source_file, df_flaw):
    """ split the given file into a list of function blocks and return their metrics into a dataframe
    """ 
    lines = list(set(list(df_flaw.Line)))
    df = pd.DataFrame()
    with open(source_file, "r") as fp:  
        liz_file = lizard.analyze_file.analyze_source_code(source_file,  fp.read())
        
        for x in range(len(liz_file.function_list)):
            
            fun_metrics = liz_file.function_list[x].__dict__
            df_fun = pd.DataFrame()
            df_fun = pd.DataFrame.from_dict(fun_metrics)
            
            start = int(fun_metrics['start_line'])
            end = int(fun_metrics['end_line']) 
            fp.seek(0) # move header to the initial point of the file

            vul_statement, cwe, vul_bool = '', [], False
            
            for l in lines:      
                code_lines = [line for line in itertools.islice(fp, start, end)]
                df_fun['code'] =  fun_metrics['long_name'] + ''.join(code_lines) 
                
                # check if the vulnerability content/statement appear in the function block or not.
                if start <= l <= end:
                    vul_bool =  True 
                    # vul_statement = vul_statement + ' \n ' +  df_flaw[df_flaw.Line==l]['Context'].values[0]
                    vul_statement = df_flaw[df_flaw.Line==l]['Context'].values[0]
                    vul_type = df_flaw[df_flaw.Line==l]['CWEs'].values[0]
                    cwe.append((vul_type, vul_statement))

                df_fun['CWEs'] = str(cwe)
                # df_fun['vul_statements'] = vul_statement
                
            df_fun['is_vul'] = vul_bool  
            df = pd.concat([df, df_fun])
            
    # <guru> I think there is a problem in lizard detecting the correct full_parameters 
    # either we have to concatenate two lines of full_parameters or ignore it and take it from long_name if needed. 
    # drop['full_parameters', 'fan_in', 'fan_out', 'general_fan_out'] because lizard has not properly 
    # implemented these parameters yet.
    
    cols_filter = ['full_parameters', 'fan_in', 'fan_out', 'general_fan_out']
    df = df.drop(cols_filter, axis=1).drop_duplicates().reset_index(drop=True)
    print('Shape of the dataframe: ', df.shape)
    return df


df_met = file2metrics(file, df)
df_met.head(3)

Shape of the dataframe:  (20, 12)


,cyclomatic_complexity,nloc,token_count,name,long_name,start_line,end_line,filename,top_nesting_level,code,CWEs,is_vul
0,3,43,336,relay_dhcp_to_server,"relay_dhcp_to_server( struct ip * ip , int len)",162,224,../data/projects/contiki-2.4/tools/tunslip.c,0,"relay_dhcp_to_server( struct ip * ip , int len)","[('CWE-120', ' memcpy(&m, inm, DHCP_BASE_LEN)...",True
1,14,91,725,relay_dhcp_to_client,relay_dhcp_to_client( int slipfd),229,338,../data/projects/contiki-2.4/tools/tunslip.c,0,relay_dhcp_to_client( int slipfd),"[('CWE-120', ' memcpy(pkt.m.options, inm.opti...",True
2,5,18,118,ip4sum,"ip4sum( u_int16_t sum , const void * _p , u_in...",344,362,../data/projects/contiki-2.4/tools/tunslip.c,0,"ip4sum( u_int16_t sum , const void * _p , u_in...",[],False


# Parsing CppCheck output:

In [44]:
import csv
from lxml import etree
import pandas as pd
import subprocess as sub


def xml2df(xml):
    """ convert xml file generated by the CppCheck tool to dataframe
    """
    df_loc = pd.read_xml(xml, encoding='utf-8', xpath='./errors/error/location')
    df_err = pd.read_xml(xml, encoding='utf-8', xpath='./errors/error')

    assert df_err['file0'].equals(df_loc['file']), 'Error! dataframe and nested location dataframe are not equals.'
    # add location metrics to the table
    df_err[['file', 'line', 'column', 'info']] = df_loc
    df_err = df_err.drop('file0', axis=1).drop_duplicates().reset_index(drop=True)
    return df_err


def cppcheck_flaws(file_or_dir):
    """ find flaws ini the file using CppCheck tool
    """
    cmd = 'cppcheck --xml ' + file_or_dir  
    process = sub.Popen(cmd,  shell=True, stdout=sub.PIPE)
    output = process.stdout.read()
    return output # xml2df(output)

# !cppcheck --template=gcc ../data/projects/contiki-2.4/apps/ 2> err.txt
# !cppcheck --template="{file}; {line}; {severity}; {message}; {code}" 
# --template-location=" {file};{line}; {info};{code}\n" ../data/projects/contiki-2.4/apps/ 2> err.txt


chk_dir = '../data/projects/contiki-2.4/apps/'
df = cppcheck_flaws(chk_dir)

chk_dir = '../data/projects/contiki-2.4/apps/'
xml = cppcheck_flaws(chk_dir)
# xtree = et.parse("students.xml")
# xroot = xtree.getroot()


<?xml version="1.0" encoding="UTF-8"?>
<results version="2">
    <cppcheck version="2.9"/>
    <errors>
        <error id="wrongPrintfScanfArgNum" severity="error" msg="sprintf format string requires 6 parameters but only 3 are given." verbose="sprintf format string requires 6 parameters but only 3 are given." cwe="685" file0="../data/projects/contiki-2.4/apps/ftp/ftpc.c">
            <location file="../data/projects/contiki-2.4/apps/ftp/ftpc.c" line="288" column="11"/>
        </error>
        <error id="overlappingWriteFunction" severity="error" msg="Overlapping read/write in memcpy() is undefined behavior" verbose="Overlapping read/write in memcpy() is undefined behavior" file0="../data/projects/contiki-2.4/apps/irc/irc.c">
            <location file="../data/projects/contiki-2.4/apps/irc/irc.c" line="129" column="3"/>
        </error>
        <error id="overlappingWriteFunction" severity="error" msg="Overlapping read/write in memcpy() is undefined behavior" verbose="Overlapping rea

In [45]:
def fetch_location(err):
    """ get locations of all the error list generated by CppCeck
    """ 
    dt_loc = {'file': [], 'line':[], 'column':[], 'info':[]}
    for loc in err.findall('location'): 
        for key, val in (loc.attrib).items():
            dt_loc[key].append(val)
    return dt_loc 
            
def xml2df(file):
    xtree = et.fromstring(open('err.xml').read())
    df = pd.DataFrame()
    for errors in xtree.findall('.//errors'):
        for err in errors.findall('error'):
            dt_err = err.attrib
            dt_err.update(fetch_location(err))
            df = pd.concat([df, pd.DataFrame([dt_err])], ignore_index=True).drop(columns=['file'], axis=1)
    return df.rename(columns={'file0':'file'})

df_flaw = xml2df('err.xml')
df_flaw.to_csv('contiki24_cppcheck.csv')

In [200]:
import itertools
import numpy as np
    
    
def file2metrics(source_file, lines, cwes, tool='cppcheck'):
    """ split the given file into a list of function blocks and return their metrics into a dataframe
    """ 
    lines = [eval(l) for l in lines] # python default casting to integer
    df = pd.DataFrame()
    
    with open(source_file, "r") as fp:  
        liz_file = lizard.analyze_file.analyze_source_code(source_file,  fp.read())
        
        for ifun in range(len(liz_file.function_list)):
            fun_metrics = liz_file.function_list[ifun].__dict__
            df_fun = pd.DataFrame.from_dict(fun_metrics)
            
            start = int(fun_metrics['start_line'])
            end = int(fun_metrics['end_line']) 
            fp.seek(0) # move header to the initial point of the file

            vul_content = ''
            vul_statements = []
            cwe  =  []
            vul_bool = False
            
            # check if any of the lines of the file belong to any functions
            for index, (l, c) in enumerate(zip(lines, cwes)):
                fun_block = [line for line in itertools.islice(fp, start, end)]
                fp.seek(0)
                df_fun['code'] =  fun_metrics['long_name'] + ''.join(fun_block) 

                # check if the vulnerability content/statement appear in the function block or not.
                # For 'is_vul' equals  True
                if start <= l <= end:
                    if tool=='cppcheck':
                        # option 1
                        vul_content = fun_block[l-start]
                        vul_statements.append(vul_content)
                        fp.seek(0)
                        
                        # option 2 - can be removed one if there is no error at the end
                        vul_stat2 = fp.readlines()[l]
            
                        assert vul_stat2==vul_content, 'Cross-check why two vul statements are different!'
                        fp.seek(0)
                    else:
                        # ToDo: take actual Context from FlawFinder's result
                        vul_content = df_flaw[df_flaw.line==l]['Context'].values[0] 
                        
                    cwe.append(c)

                df_fun['fun_name'] = fun_metrics['name']
                df_fun['content'] = str(vul_statements) if len(vul_statements)>0 else ''
                df_fun['is_vul'] = True if cwe else False 
                
                if len(cwe)>0:
                    # df_fun['cwe'] = 'unknown'  if np.isnan(cwe).all() else str(cwe)
                    df_fun['cwe'] = 'unknown' if all(i != i for i in cwe) else str(cwe)
                else:
                    df_fun['cwe'] = 'benign'
          
            df = pd.concat([df, df_fun])

    # <guru> I think there is a problem in lizard detecting the correct full_parameters 
    # either we have to concatenate two lines of full_parameters or ignore it and take it from long_name if needed. 
    # drop['full_parameters', 'fan_in', 'fan_out', 'general_fan_out'] because lizard has not properly 
    # implemented these parameters yet.
    
    cols_filter = ['full_parameters', 'fan_in', 'fan_out', 'general_fan_out', 'top_nesting_level']
    df = df.drop(cols_filter, axis=1).drop_duplicates().reset_index(drop=True)
    return df

df_prj = pd.DataFrame()

for x in list(set(df_flaw.file)): # on every unique files
    lines = list(df_flaw[df_flaw.file==x]['line'])
    cwes = list(df_flaw[df_flaw.file==x]['cwe'])
    # vul_statements = list(df_flaw[df_flaw.file==x]['cwe'])
    lines = [x[0] if len(x)==1 else [x[0], x[1]] for x in lines]
    
    df_file = file2metrics(x, lines, cwes, tool='cppcheck')
    df_prj = pd.concat([df_prj, df_file])
    
df_prj = df_prj.reset_index(drop=True).drop_duplicates()
df_prj

,cyclomatic_complexity,nloc,token_count,name,long_name,start_line,end_line,filename,code,fun_name,content,is_vul,cwe
0,1,8,34,quit,quit( void),105,112,../data/projects/contiki-2.4/apps/irc/irc.c,quit( void){\n ctk_window_close(&window);\n ...,quit,,False,benign
1,6,32,223,ircc_text_output,"ircc_text_output( struct ircc_state * s , char...",115,154,../data/projects/contiki-2.4/apps/irc/irc.c,"ircc_text_output( struct ircc_state * s , char...",ircc_text_output,"['\n', ' strncpy(&log[LOG_WIDTH * (LOG_HEIG...",True,unknown
2,8,32,315,parse_line,parse_line( void),157,190,../data/projects/contiki-2.4/apps/irc/irc.c,parse_line( void){\n int i;\n for(i = 0; i <...,parse_line,,False,benign
3,1,5,21,ircc_sent,ircc_sent( struct ircc_state * s),193,198,../data/projects/contiki-2.4/apps/irc/irc.c,ircc_sent( struct ircc_state * s){\n /* ctk_...,ircc_sent,,False,benign
4,20,77,525,PROCESS_THREAD,"PROCESS_THREAD( irc_process , ev , data)",200,295,../data/projects/contiki-2.4/apps/irc/irc.c,"PROCESS_THREAD( irc_process , ev , data){\n c...",PROCESS_THREAD,,False,benign
5,1,4,18,ircc_closed,ircc_closed( struct ircc_state * s),298,301,../data/projects/contiki-2.4/apps/irc/irc.c,ircc_closed( struct ircc_state * s){\n ircc_t...,ircc_closed,,False,benign
6,1,4,18,ircc_connected,ircc_connected( struct ircc_state * s),304,307,../data/projects/contiki-2.4/apps/irc/irc.c,ircc_connected( struct ircc_state * s){\n irc...,ircc_connected,,False,benign
7,13,53,294,PROCESS_THREAD,"PROCESS_THREAD( shell_blink_process , ev , data)",55,115,../data/projects/contiki-2.4/apps/shell/shell-...,"PROCESS_THREAD( shell_blink_process , ev , dat...",PROCESS_THREAD,['\n'],True,unknown
8,1,4,12,shell_blink_init,shell_blink_init( void),118,121,../data/projects/contiki-2.4/apps/shell/shell-...,shell_blink_init( void){\n shell_register_com...,shell_blink_init,,False,benign
9,1,15,121,sniff_attributes_output,sniff_attributes_output( int type),77,92,../data/projects/contiki-2.4/apps/shell/shell-...,sniff_attributes_output( int type){\n struct ...,sniff_attributes_output,,False,benign
